In [1]:
# import packages
from collections import deque
import pandas as pd
import numpy as np
import random
import copy
from collections import Counter

In [2]:
def find_free_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy):
    
    ###########################################################
    #NOTE:男孩和女孩的序号可能并不规整，例如1，3，5，7，8，9，11],特别是离婚之后的情形
    #     这时候进行转换为[1,2,3,4,5,6,7]
    ###########################################################
    boy_index_dict = {}
    for i in range(len(boys)):
        boy_index_dict[boys[i]] = i+1
    girl_index_dict = {}
    for i in range(len(girls)):
        girl_index_dict[girls[i]] = i+1
    # old_boys = boys
    # old_girls = girls
    new_boys = list(range(1,len(boys)+1))
    new_girls = list(range(1,len(girls)+1))
    new_sort_boy_to_girl = []
    for items in sort_boy_to_girl:
        tmp = []
        for item in items:
            tmp.append(girl_index_dict[item])
        new_sort_boy_to_girl.append(tmp)
    new_sort_girl_to_boy = []
    for items in sort_girl_to_boy:
        tmp = []
        for item in items:
            tmp.append(boy_index_dict[item])
        new_sort_girl_to_boy.append(tmp)
    ###########################################################
    # print(girl_index_dict)
    # 当前选项
    # 建立字典：key：第n个男生，value：当前选择，current_boys = {boys[0]:None, boys[1]:None, boys[2]:None, boys[3]:None ... }
    current_boys = dict(zip(new_boys, [0]*len(new_boys)))
    # 建立字典：key：第n个女生，value：当前选择，current_girls = {girls[0]:None, girls[1]:None, girls[2]:None, boys[3]:None ... }
    current_girls = dict(zip(new_girls, [0]*len(new_girls)))
    # sumboys= sum(current_girls)
    count = len(boys)
 
 
    # 建立字典：男孩下一次将要选择的女孩
    next_select = dict(zip(new_boys, [0]*len(new_boys)))
    # 建立队列，对每一个男生
    for i in range(count):
        # 向排序的第一个女生求婚
        temp = [new_girls[m-1] for m in new_sort_boy_to_girl[i]]
        # 下次选择排序的第二个女生
        next_select[new_boys[i]] = deque(temp)
 
 
    # 女选男字典
    sort_girl = dict(zip(new_girls, [0]*len(new_boys)))
    for i in range(count):
        # 女孩对男孩排名，通过题目给出的sort_girl_to_boy字典,排在前面的名字好感度比较高 
        temp = [[new_boys[m-1], 19-ind] for ind, m in enumerate(new_sort_girl_to_boy[i])]
        name = []
        match = []
        # 对每个女生而言
        for t in temp:
            # 匹配的男生序号
            name.append(t[0])
            # 匹配的男生好感度
            match.append(t[1])
        # 得到女生目前排序
        sort_girl[new_girls[i]] = dict(zip(name, match))
 
 
    times = 0
    # while None in current_boys.values():    
    while times < 3:
        times += 1
        for i in range(count):
            # 第i个男孩求婚
            bid = new_boys[i]
            if current_boys[bid]:
                # 男孩有对象，跳过
                continue
            else:
                # 选择排序的下一个
                select = next_select[bid][0]
                # 如果女孩没对象
                if current_girls[select] == 0:
                    #两者结合
                    current_boys[bid] = select
                    current_girls[select] = bid
                    next_select[bid].popleft()
                else:
                    # 和女孩的对象好感度对比,如果对现任的好感度,大于第三者,不动
                    if sort_girl[select][current_girls[select]] > sort_girl[select][bid]:
                        next_select[bid].popleft()
                    # 如果与上面相反
                    # 现任男失恋,第三者男选择了当前女,当前女选择了第三者男
                    # 现任男失去对当前女孩的追求权
                    else:
                        current_boys[current_girls[select]] = 0
                        current_boys[bid] = select
                        current_girls[select] = bid
                        next_select[bid].popleft()
    print(current_boys,current_girls)
    #NOTE:还原序号
    return_boys = dict()
    for key,value in current_boys.items():
        if value==0:
            return_boys[boys[key-1]] = 0
        else:
            return_boys[boys[key-1]] = girls[value-1]
    return return_boys

In [3]:
#NOTE:new add
def count_blocking_pairs(match_dict,sort_boy_to_girl, sort_girl_to_boy):
    tr_match_dict = {i:0 for i in range(len(match_dict))}
    for key,value in match_dict.items():
        if value:
            tr_match_dict[value]=key

    count = 0
    for key,value in match_dict.items():
        if value==0: #没匹配上
            continue
        else:
            miss_match = False
            boy_to_girl_index = sort_boy_to_girl[key-1].index(value) #目前匹配的女生的排序
            if boy_to_girl_index!=0: #不是最高时，遍历所有更高排序的女生
                for i in range(boy_to_girl_index):
                    girl_index = sort_boy_to_girl[key-1][i] # 目前遍历到的女生
                    girl_current_boy_index = tr_match_dict[girl_index] # 目前女生的匹配男生
                    #该女生未匹配到人，或者匹配的人不如男生
                    if girl_current_boy_index == 0: 
                        miss_match = True
                        break
                    elif sort_girl_to_boy[girl_index-1].index(key)<sort_girl_to_boy[girl_index-1].index(girl_current_boy_index):
                        miss_match = True
                        break
                if miss_match:
                    continue
            
            girl_to_boy_index = sort_girl_to_boy[value-1].index(key)
            if girl_to_boy_index!=0: #不是最高时，遍历所有更高排序的男生
                for i in range(girl_to_boy_index):
                    boy_index = sort_girl_to_boy[value-1][i] # 目前遍历到的男生
                    boy_current_girl_index = match_dict[boy_index] # 目前男生的匹配女生
                    #该男生未匹配到人，或者匹配的人不如女生
                    if boy_current_girl_index == 0: 
                        miss_match = True
                        break
                    elif sort_boy_to_girl[boy_index-1].index(value)<sort_boy_to_girl[boy_index-1].index(boy_current_girl_index):
                        miss_match = True
                        break
                if miss_match:
                    continue
            count += 1
    return count

In [4]:
## 初始化
size=20
boys = list(range(1,size+1))
girls = list(range(1,size+1))
# rep: repeated time

In [5]:
# 1-20随机排序生成男对女排序列表
sort_boy_to_girl= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
# print(sort_boy_to_girl)
# 把男对女排序列表作为一个dataframe打印
dfboys = pd.DataFrame(pd.Series(sort_boy_to_girl),columns={'boy_preferece'})
# 修改序号，从1开始
dfboys.index=dfboys.index+1
dfboys=dfboys.reset_index().rename(columns = {'index':'boy'})

In [6]:
# 1-20随机排序生成女对男排序列表
sort_girl_to_boy= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
# print(sort_girl_to_boy)
# 把女对男排序列表作为一个dataframe打印
dfgirls = pd.DataFrame(pd.Series(sort_girl_to_boy),columns={'girl_preference'})
# 修改序号，女从1开始
dfgirls.index=dfgirls.index+1
dfgirls=dfgirls.reset_index().rename(columns = {'index':'girl'})
# dfgirls

In [7]:
# 打印第一次排序结果
result = find_free_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy)
df = pd.DataFrame(pd.Series(result), columns=['girl_match'])
df=df.reset_index().rename(columns = {'index':'boy_match'})
#NOTE:test
print(count_blocking_pairs(result,sort_boy_to_girl, sort_girl_to_boy))
df

{1: 12, 2: 15, 3: 5, 4: 3, 5: 4, 6: 0, 7: 16, 8: 9, 9: 19, 10: 11, 11: 7, 12: 6, 13: 2, 14: 0, 15: 17, 16: 13, 17: 0, 18: 14, 19: 0, 20: 18} {1: 0, 2: 13, 3: 4, 4: 5, 5: 3, 6: 12, 7: 11, 8: 0, 9: 8, 10: 0, 11: 10, 12: 1, 13: 16, 14: 18, 15: 2, 16: 7, 17: 15, 18: 20, 19: 9, 20: 0}
9


,boy_match,girl_match
0,1,12
1,2,15
2,3,5
3,4,3
4,5,4
5,6,0
6,7,16
7,8,9
8,9,19
9,10,11


In [8]:
# mr_girl_sum: 女生匹配到的男生的排名加和
# mr_boy_sum: 男生匹配到的女生的排名加和
mr_girl_sum=0
mr_boy_sum=0

# amr_girl: 女生匹配到的男生的排名平均 (mr_girl_sum/rep)
# amr_boy: 男生匹配到的女生的排名平均 (mr_boy_sum/rep)
amr_girl=0
amr_boy=0
# repeat

In [9]:
# 计算单次
# 计算男生匹配到的女生的排名平均 (mr_boy_sum/rep)
girl_match=pd.Series(find_free_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy))
mr_boy_sum=0
for i in range(1,size):
    mr_boy=sort_boy_to_girl[i-1].index(girl_match[i])
   # print("mr_boy=",mr_boy)
    mr_boy_sum+=mr_boy+1
    # print("mr_boy_sum=",mr_boy_sum)
    i+=1
# print(mr_boy_sum)
amr_boy=mr_boy_sum/size
print("amr_boy=",amr_boy)

# 计算女生匹配到的男生的排名平均 (mr_girl_sum/rep)
mr_girl_sum=0
for i in range(1,size):
    mr_girl=sort_girl_to_boy[i-1].index(i)
    mr_girl_sum+=mr_girl
    i+=1
amr_girl=mr_girl_sum/size
print("amr_girl=",amr_girl)

{1: 12, 2: 15, 3: 5, 4: 3, 5: 4, 6: 0, 7: 16, 8: 9, 9: 19, 10: 11, 11: 7, 12: 6, 13: 2, 14: 0, 15: 17, 16: 13, 17: 0, 18: 14, 19: 0, 20: 18} {1: 0, 2: 13, 3: 4, 4: 5, 5: 3, 6: 12, 7: 11, 8: 0, 9: 8, 10: 0, 11: 10, 12: 1, 13: 16, 14: 18, 15: 2, 16: 7, 17: 15, 18: 20, 19: 9, 20: 0}


ValueError: 0 is not in list

In [10]:
# 重复rep次
rep = 10 #add
for i in range(1,rep):
    # 设定随机排序矩阵
    # 男对女排序
    sort_boy_to_girl= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
    # 女对男排序
    sort_girl_to_boy= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
    # 运行算法
    girl_match=pd.Series(find_free_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy))
    # 一次
    for i in range(1,size):
        # 当前男生对女生排序
        mr_boy=sort_boy_to_girl[i-1].index(girl_match[i])+1
        # 当前女生对男生排序
        mr_girl=sort_girl_to_boy[i-1].index(i)
        # 男生对女生排序累加
        mr_boy_sum += mr_boy
        # 女生对男生排序累加
        mr_girl_sum += mr_girl
        i+=1
    # 重复计算男
    amr_boy+=mr_boy_sum/size/rep
    # 重复计算女
    amr_girl+=mr_girl_sum/size/rep
    i+=1
print("amr_boy=",amr_boy/rep)
print("amr_girl=",amr_girl/rep)
print("amr_total=",amr_boy/rep/2+amr_girl/rep/2)
print("fe=",abs(amr_boy/rep/2-amr_girl/rep/2))

{1: 16, 2: 10, 3: 4, 4: 9, 5: 6, 6: 0, 7: 15, 8: 17, 9: 3, 10: 5, 11: 14, 12: 7, 13: 8, 14: 0, 15: 18, 16: 0, 17: 1, 18: 11, 19: 20, 20: 0} {1: 17, 2: 0, 3: 9, 4: 3, 5: 10, 6: 5, 7: 12, 8: 13, 9: 4, 10: 2, 11: 18, 12: 0, 13: 0, 14: 11, 15: 7, 16: 1, 17: 8, 18: 15, 19: 0, 20: 19}


ValueError: 0 is not in list

In [10]:
# 拆散概率
prob=0.6
size=20

# 假设拆散的男生为1-20中抽取20*0.6个
crossover= random.sample(range(1,size+1),12)
# crossover

In [11]:
# 定义从list中删除list的函数
def diff_of_two_list(lst1, lst2): 
    diff_list = []
    for item in lst1:
        if item not in lst2:
            diff_list.append(item) 
    return diff_list

In [12]:
# 拆出的男生和女生
df2=df[df["boy_match"].isin(crossover)]
# df2
# 得到离婚男生list
boy_divorce=df2['boy_match'].values.tolist()
boy_divorce.sort()
# 离婚女生list
girl_divorce=df2['girl_match'].values.tolist()
# boy_divorce
girl_divorce.sort()
# girl_divorce

In [13]:
# 未离婚男女（要从列表中删去）
girl_not_divorce=diff_of_two_list(girls,girl_divorce)
boy_not_divorce=diff_of_two_list(boys,boy_divorce)
# girl_not_divorce

In [14]:
# 离婚男对离婚女排序
if __name__ == '__main__':
    sort_boy_to_girl_divorce = []
    # for i in range(20):#NOTE:这里1到12没法儿筛出离婚男吧
    for i in boy_divorce:#NOTE:这里1到12没法儿筛出离婚男吧
        # 男对女排序 减去 不在场上的女
        item=diff_of_two_list(sort_boy_to_girl[i-1],girl_not_divorce)
        sort_boy_to_girl_divorce.append(item) 
# sort_boy_to_girl_divorce

In [15]:
# 离婚女对离婚男排序
if __name__ == '__main__':
    sort_girl_to_boy_divorce = []
    # for i in range(20):#NOTE：same
    for i in girl_divorce:
        # 女对男排序 减去 不在场上的男
        item=diff_of_two_list(sort_girl_to_boy[i-1],boy_not_divorce)
        sort_girl_to_boy_divorce.append(item) 
# sort_girl_to_boy_divorce

In [16]:
divorce_match=pd.Series(find_free_partner(boy_divorce, girl_divorce, sort_boy_to_girl_divorce, sort_girl_to_boy_divorce))
divorce_match=pd.DataFrame(list(divorce_match.items()),columns=['boy', 'girl'])
# divorce_match

In [17]:
# 计算单次
# 计算男生匹配到的女生的排名平均 (mr_boy_sum/rep)
girl_match_divorce=pd.Series(find_free_partner(boy_divorce, girl_divorce, sort_boy_to_girl_divorce, sort_girl_to_boy_divorce))
mr_boy_sum=0

for i in range(1,12):
    mr_boy=sort_boy_to_girl_divorce[i].index(divorce_match.iloc[i,1])
    mr_boy_sum+=mr_boy+1
    i+=1
amr_boy=mr_boy_sum/size
print("amr_boy=",amr_boy)

# 计算女生匹配到的男生的排名平均 (mr_girl_sum/rep)
mr_girl_sum=0
for i in range(1,12):
    mr_girl=sort_girl_to_boy_divorce[i].index(divorce_match.iloc[i,0])
    mr_girl_sum+=mr_girl
    i+=1
amr_girl=mr_girl_sum/size
print("amr_girl=",amr_girl)

amr_boy= 1.9
amr_girl= 4.05


In [18]:
# 重复rep次
rep = 10 
for i in range(1,rep):
    # 随机得到离婚男性
    crossover= random.sample(range(1,size+1),12)
    # 离婚男女组合
    df2=df[df["boy_match"].isin(crossover)]
    # df2
    # 离婚男list
    boy_divorce=df2['boy_match'].values.tolist()
    boy_divorce.sort()
    boy_not_divorce=diff_of_two_list(boys,boy_divorce)
     # 离婚女list
    girl_divorce=df2['girl_match'].values.tolist()
    girl_divorce.sort()
    girl_not_divorce=diff_of_two_list(girls,girl_divorce)

    # 离婚男对离婚女排序
    if __name__ == '__main__':
        sort_boy_to_girl_divorce = []
    # for i in range(20)
        for i in boy_divorce:
        # 删去不在场上的
            item=diff_of_two_list(sort_boy_to_girl[i-1],girl_not_divorce)
            sort_boy_to_girl_divorce.append(item) 
    sort_boy_to_girl_divorce
    #离婚女对离婚男排序
    if __name__ == '__main__':
        sort_girl_to_boy_divorce = []
    # for i in range(20):#NOTE：same
        for i in girl_divorce:
        # 删去不在场上的
            item=diff_of_two_list(sort_girl_to_boy[i-1],boy_not_divorce)
            sort_girl_to_boy_divorce.append(item) 
            sort_girl_to_boy_divorce

# 一次
# 计算 amr_boy (mr_boy_sum/rep)
    divorce_match=find_free_partner(boy_divorce, girl_divorce, sort_boy_to_girl_divorce, sort_girl_to_boy_divorce)
    divorce_match=pd.DataFrame(list(divorce_match.items()),columns=['boy', 'girl'])
    for i in range(0,11):
        mr_boy=sort_boy_to_girl[divorce_match.iloc[i,0]].index(divorce_match.iloc[i,1])+1
        mr_boy_sum+=mr_boy
        i+=1
    amr_boy=mr_boy_sum/size
    print("amr_boy=",amr_boy/rep)
    
    for i in range(0,5):
        mr_girl=sort_girl_to_boy[divorce_match.iloc[i,1]].index(divorce_match.iloc[i,0])+1
        mr_girl_sum+=mr_girl
        i+=1
        amr_girl=mr_girl_sum/size
    print("amr_girl=",amr_girl/rep)
    print("fe=",amr_boy/rep/2-amr_girl/rep/2)

    # 计算重复平均，average rank of matched girl for boys
    amr_boy+=mr_boy_sum/size/rep
    # 计算重复平均，average rank of matched boy for girls, in rep
    amr_girl+=mr_girl_sum/size/rep
    i+=1
print("amr_boy=",amr_boy/rep)
print("amr_girl=",amr_girl/rep)
print("fe=",amr_boy/rep/2-amr_girl/rep/2)
print("amr_total=",amr_boy/rep/2+amr_girl/rep/2)
print("fe=",amr_boy/rep/2-amr_girl/rep/2)



amr_boy= 0.7849999999999999
amr_girl= 0.61
fe= 0.08749999999999997
amr_boy= 1.45
amr_girl= 0.86
fe= 0.295
amr_boy= 2.07
amr_girl= 1.03
fe= 0.5199999999999999
amr_boy= 2.6399999999999997
amr_girl= 1.3199999999999998
fe= 0.6599999999999999
amr_boy= 3.4299999999999997
amr_girl= 1.54
fe= 0.9449999999999998
amr_boy= 4.085
amr_girl= 1.72
fe= 1.1825
amr_boy= 4.779999999999999
amr_girl= 1.9449999999999998
fe= 1.4174999999999998
amr_boy= 5.3549999999999995
amr_girl= 2.1100000000000003
fe= 1.6224999999999996
amr_boy= 6.055
amr_girl= 2.285
fe= 1.8849999999999998
amr_boy= 6.660499999999999
amr_girl= 2.5135
fe= 2.0734999999999992
amr_total= 4.587
fe= 2.0734999999999992
